<a href="https://colab.research.google.com/github/Birat-Poudel/Conversational-Computational-Empathy/blob/master/Transformer_Intent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data=pd.read_csv("/content/drive/MyDrive/test123.csv", encoding="latin-1")

In [ ]:
data.head()

,customer_query,intent,sentiment,responses,Unnamed: 4,Unnamed: 5
0,The noodles I purchased have a strange chemica...,complaint,negative,We apologize for the noodles with a chemical s...,NaN,NaN
1,The shoes I bought have a loose sole. It feels...,complaint,negative,We're sorry for the discolored strap. Please r...,NaN,NaN
2,The refrigerator I purchased has a dent on the...,complaint,negative,We apologize for the inconvenience. Please sha...,NaN,NaN
3,The slippers I received have a loose insole th...,complaint,negative,We apologize for the rough interior lining. Pl...,NaN,NaN
4,The necklace I received looks significantly di...,complaint,negative,Understood. Please provide us with your order ...,NaN,NaN


In [ ]:
data.shape

(10200, 6)

In [ ]:
one_hot = pd.get_dummies(data['intent'])
data.drop(['intent','sentiment','responses','Unnamed: 4', 'Unnamed: 5'], axis=1, inplace=True)
data = pd.concat([data, one_hot], axis=1)
data.head()

,customer_query,complaint,feedback,inquiry,refund
0,The noodles I purchased have a strange chemica...,1,0,0,0
1,The shoes I bought have a loose sole. It feels...,1,0,0,0
2,The refrigerator I purchased has a dent on the...,1,0,0,0
3,The slippers I received have a loose insole th...,1,0,0,0
4,The necklace I received looks significantly di...,1,0,0,0


In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [
                layers.Dense(ff_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    # removing contractions
    sentence = re.sub(r"i'm", "i am", sentence)
    sentence = re.sub(r"he's", "he is", sentence)
    sentence = re.sub(r"she's", "she is", sentence)
    sentence = re.sub(r"it's", "it is", sentence)
    sentence = re.sub(r"that's", "that is", sentence)
    sentence = re.sub(r"what's", "that is", sentence)
    sentence = re.sub(r"where's", "where is", sentence)
    sentence = re.sub(r"how's", "how is", sentence)
    sentence = re.sub(r"\'ll", " will", sentence)
    sentence = re.sub(r"\'ve", " have", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"\'d", " would", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"won't", "will not", sentence)
    sentence = re.sub(r"can't", "cannot", sentence)
    sentence = re.sub(r"n't", " not", sentence)
    sentence = re.sub(r"n'", "ng", sentence)
    sentence = re.sub(r"'bout", "about", sentence)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
    sentence = sentence.strip()
    return sentence

In [ ]:
X = data["customer_query"].values
y = data.drop(["customer_query"], axis=1).values

max_features = 20000
maxlen = 150
vocab_size = 20000

tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=150)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42
)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(7140, 150) (7140, 4)
(3060, 150) (3060, 4)


In [ ]:
import pickle

# Save the tokenizer to a file using pickle
with open('/content/drive/MyDrive/Major/tokenizer/intent_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print(y)

[[0 0 0 1]
 [0 0 0 1]
 [1 0 0 0]
 ...
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]]


In [ ]:
embed_dim = 100
num_heads = 2
ff_dim = 100

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(4, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

In [ ]:
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
history = model.fit(
    X_train, y_train, batch_size=128, epochs=10, validation_data=(X_test, y_test), callbacks=[early_stopping]
)

Epoch 1/10
56/56 [==============================] - 61s 1s/step - loss: 1.3925 - accuracy: 0.2801 - val_loss: 1.3508 - val_accuracy: 0.4500
Epoch 2/10
56/56 [==============================] - 57s 1s/step - loss: 1.1474 - accuracy: 0.4714 - val_loss: 0.6705 - val_accuracy: 0.7422
Epoch 3/10
56/56 [==============================] - 61s 1s/step - loss: 0.3513 - accuracy: 0.8718 - val_loss: 0.1466 - val_accuracy: 0.9601
Epoch 4/10
56/56 [==============================] - 56s 999ms/step - loss: 0.1067 - accuracy: 0.9713 - val_loss: 0.1144 - val_accuracy: 0.9752
Epoch 5/10
56/56 [==============================] - 57s 1s/step - loss: 0.0601 - accuracy: 0.9850 - val_loss: 0.1078 - val_accuracy: 0.9794
Epoch 6/10
56/56 [==============================] - 59s 1s/step - loss: 0.0426 - accuracy: 0.9899 - val_loss: 0.1088 - val_accuracy: 0.9794
Epoch 7/10
56/56 [==============================] - 58s 1s/step - loss: 0.0362 - accuracy: 0.9912 - val_loss: 0.1084 - val_accuracy: 0.9820
Epoch 8/10
56/56 

In [ ]:
model.save("/content/drive/MyDrive/Major/IntentClassificationTransformer1.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
text = ["How can I track my order?"]
# How can I track my order?

text = preprocess_sentence(text[0])

text = tokenizer.texts_to_sequences(text)
text = pad_sequences(text, maxlen=150, dtype='int32', value=0)
intent = model.predict(text,batch_size=1,verbose = 2)[0]

print(intent)

if (np.argmax(intent) == 0):
  print("complaint")
elif (np.argmax(intent) == 1):
  print("feedback")
elif (np.argmax(intent) == 2):
  print("inquiry")
else:
  print("refund")

26/26 - 0s - 180ms/epoch - 7ms/step
[2.7786658e-04 4.5732982e-03 9.9404895e-01 1.0999435e-03]
inquiry
